# Binary Diagnostic

## Part 1

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:
```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```
Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.

So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. **What is the power consumption of the submarine?** (Be sure to represent your answer in decimal, not binary.)

In [1]:
from IPython.display import Markdown
import numpy as np

In [2]:
test_input = [
    '00100',
    '11110',
    '10110',
    '10111',
    '10101',
    '01111',
    '00111',
    '11100',
    '10000',
    '11001',
    '00010',
    '01010']

In [3]:
bitsum = np.zeros(12, dtype=np.int)

nlines = 0
with open('input.txt', 'r') as fid:
    for lnum, line in enumerate(fid):
        bitstr = line.strip()
        bitarr = np.array([int(x) for x in line.strip()], dtype=np.int)
        bitsum += bitarr
nlines = lnum + 1

In [4]:
# Wherever bitsum exceeds nlines // 2 there are more 1's than zeros
gamma_bin = ''.join([str(x) for x in np.array(bitsum > nlines // 2, dtype=np.int).tolist()])
gamma = int(gamma_bin, base=2)
#display(gamma)

# For the least common bit we just need to invert gamma_bin
epsilon_bin = ''.join([str(y) for y in map(lambda x: int(not(int(x))), gamma_bin)])
epsilon = int(epsilon_bin, base=2)
#display(epsilon)

#Markdown("The power consumption is: **{}**".format(gamma * epsilon))

## Part Two

Next, you should verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

- Keep only numbers selected by the bit criteria for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
- If you only have one number left, stop; this is the rating value for which you are searching.
- Otherwise, repeat the process, considering the next bit to the right.

The bit criteria depends on which type of rating value you want to find:

- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

For example, to determine the oxygen generator rating value using the same example diagnostic report from above:

- Start with all 12 numbers and consider only the first bit of each number. There are more 1 bits (7) than 0 bits (5), so keep only the 7 numbers with a 1 in the first position: 11110, 10110, 10111, 10101, 11100, 10000, and 11001.
- Then, consider the second bit of the 7 remaining numbers: there are more 0 bits (4) than 1 bits (3), so keep only the 4 numbers with a 0 in the second position: 10110, 10111, 10101, and 10000.
- In the third position, three of the four numbers have a 1, so keep those three: 10110, 10111, and 10101.
- In the fourth position, two of the three numbers have a 1, so keep those two: 10110 and 10111.
- In the fifth position, there are an equal number of 0 bits and 1 bits (one each). So, to find the oxygen generator rating, keep the number with a 1 in that position: 10111.
- As there is only one number left, stop; the oxygen generator rating is 10111, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

- Start again with all 12 numbers and consider only the first bit of each number. There are fewer 0 bits (5) than 1 bits (7), so keep only the 5 numbers with a 0 in the first position: 00100, 01111, 00111, 00010, and 01010.
- Then, consider the second bit of the 5 remaining numbers: there are fewer 1 bits (2) than 0 bits (3), so keep only the 2 numbers with a 1 in the second position: 01111 and 01010.
- In the third position, there are an equal number of 0 bits and 1 bits (one each). So, to find the CO2 scrubber rating, keep the number with a 0 in that position: 01010.
- As there is only one number left, stop; the CO2 scrubber rating is 01010, or 10 in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. **What is the life support rating of the submarine?** (Be sure to represent your answer in decimal, not binary.)

In [5]:
# Get the data all into memory in a list for easier processing
dat = list()
with open('input.txt', 'r') as fid:
    for line in fid:
        dat.append(line.strip())
#dat

In [6]:
def get_common(strlist, bitpos, mode):
    # For the supplied list of bit strings, return the
    # 'most' or 'least' common bit at bitpos depending on the mode
    bitsum = 0
    nlines = len(strlist)
    for bitstr in strlist:
        #print(bitstr[bitpos])
        bitsum += int(bitstr[bitpos])
    #print('bitsum: {}; nlines / 2.: {}'.format(bitsum, nlines / 2.))
    if -1e-5 < bitsum - nlines / 2.:
        # 1 is most common or 0's and 1's are equally common
        if 'most' == mode:
            #print('1 is most common')
            return 1
        elif 'least' == mode:
            #print('0 is least common')
            return 0
        else:
            raise ValueError("Unhandled mode: {}",format(mode))
    else:
        # 0 is most common
        if 'most' == mode:
            #print('0 is most common')
            return 0
        elif 'least' == mode:
            #print('1 is least common')
            return 1
        else:
            raise ValueError("Unhandled mode: {}",format(mode))


def filter_for(bitlist, bitval, bitpos):
    # Return list for bitstrs of bitlist with bitval at bitpos
    return [x for x in bitlist if str(x[bitpos]) == str(bitval)]

def get_rating(bitlist, mode):
    # Return rating of bitlist for mode of 'most' or 'least'
    niters = len(bitlist[0])
    for i in range(niters):
        bitval = get_common(bitlist, i, mode=mode)
        #print("{}: {}".format(bitval, bitlist))
        bitlist = filter_for(bitlist, bitval, i)
        if 2 > len(bitlist):
            break
    #bitlist
    rating = int(bitlist[0], base=2)
    return rating

In [7]:
#get_common(test_input, 1, mode='most')
#filter_for(test_input, 0, 1)

In [8]:
oxrating = get_rating(dat, mode='most')
cdioxrating = get_rating(dat, mode='least')
lifesupport = oxrating * cdioxrating
#Markdown("Life support rating is: **{}**".format(lifesupport))